In [37]:
import xml.etree.ElementTree as ET
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

In [38]:
Posts_tree = ET.parse('Data_Dump/Posts.xml')
Posts_root = Posts_tree.getroot()
len(Posts_root)

7347

In [39]:
for child in Posts_root[:5]:
    print(child.attrib)

{'Id': '1', 'PostTypeId': '1', 'CreationDate': '2010-09-01T19:32:45.710', 'Score': '32', 'ViewCount': '1815', 'Body': "<p>I'm looking at the questions proposed during the Area 51 process:</p>\n\n<ul>\n<li>My supervisor thinks that all <code>If</code> statements should include <code>else</code> statements. Do you agree?</li>\n<li>What are common mistakes in Software Development?</li>\n<li>Tabs vs. Spaces: What is the one proper indentation character for everything, in every situation, ever?</li>\n<li>What programming language should I teach to my 4 year old son?</li>\n<li>What was the turning point of your programming career?</li>\n</ul>\n\n<p>None of these have an answer that should be accepted. The questions are interesting, and the answers would also be informative if the answer was well written and explained why the answerer thinks his method or idea is better. But I can't really see being able to accept an answer to any of these questions.</p>\n\n<p>So, if I ask a question, how do 

In [40]:
df = pd.DataFrame(columns = ['Title', 'Body'])

In [57]:
posts = []
for child in Posts_root:
    posts.append(child.attrib)
print(posts[0])

{'Id': '1', 'PostTypeId': '1', 'CreationDate': '2010-09-01T19:32:45.710', 'Score': '32', 'ViewCount': '1815', 'Body': "<p>I'm looking at the questions proposed during the Area 51 process:</p>\n\n<ul>\n<li>My supervisor thinks that all <code>If</code> statements should include <code>else</code> statements. Do you agree?</li>\n<li>What are common mistakes in Software Development?</li>\n<li>Tabs vs. Spaces: What is the one proper indentation character for everything, in every situation, ever?</li>\n<li>What programming language should I teach to my 4 year old son?</li>\n<li>What was the turning point of your programming career?</li>\n</ul>\n\n<p>None of these have an answer that should be accepted. The questions are interesting, and the answers would also be informative if the answer was well written and explained why the answerer thinks his method or idea is better. But I can't really see being able to accept an answer to any of these questions.</p>\n\n<p>So, if I ask a question, how do 

In [58]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

In [59]:
for i, post in enumerate(posts):
    if 'Title' in post:
        df = df.append({'Title': post['Title'], 'Body': remove_html_tags(post['Body'])}, ignore_index=True)

In [60]:
df.head(6)

,Title,Body
0,Why would anyone accept an answer?,I'm looking at the questions proposed during t...
1,What should our FAQ contain?,One of the big 7 questions.\n
2,What should our domain name be?,\nPossible Duplicate:\nWrite an Elevator Pitch...
3,What should our logo and site design look like?,One of the big 7 questions.\n\n\nOne suggestio...
4,Who should the moderators be?,\n Possible Duplicate:\n Moderator Pro Tem A...
5,Why would anyone accept an answer?,I'm looking at the questions proposed during t...


In [46]:
## tokenize, stem and lemmatize

for i, j in df.iterrows(): 
    title_tokens_stemmed = [ps.stem(word) for word in word_tokenize(j["Title"])]
    title_tokens_stemmed_lemmetized = [lemmatizer.lemmatize(word) for word in title_tokens_stemmed]
    title_tokens_stemmed_lemmetized = ' '.join(title_tokens_stemmed_lemmetized)
    df.at[i, "Title"] = title_tokens_stemmed_lemmetized
    
    body_tokens_stemmed = [ps.stem(word) for word in word_tokenize(j["Body"])]
    body_tokens_stemmed_lemmetized = [lemmatizer.lemmatize(word) for word in body_tokens_stemmed]
    body_tokens_stemmed_lemmetized = ' '.join(body_tokens_stemmed_lemmetized)
    df.at[i, "Body"] = body_tokens_stemmed_lemmetized


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/mitagavade/nltk_data'
    - '/Users/mitagavade/Downloads/flair demo/venvai/nltk_data'
    - '/Users/mitagavade/Downloads/flair demo/venvai/share/nltk_data'
    - '/Users/mitagavade/Downloads/flair demo/venvai/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [61]:
df.head(5)

,Title,Body
0,Why would anyone accept an answer?,I'm looking at the questions proposed during t...
1,What should our FAQ contain?,One of the big 7 questions.\n
2,What should our domain name be?,\nPossible Duplicate:\nWrite an Elevator Pitch...
3,What should our logo and site design look like?,One of the big 7 questions.\n\n\nOne suggestio...
4,Who should the moderators be?,\n Possible Duplicate:\n Moderator Pro Tem A...


In [62]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

In [91]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
 
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis


def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}: {}'.format(topic_idx, ' | '.join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]])))
        
number_topics = 10
number_words = 10


# fit the LDA model on all of the question titles for the data
post_titles = count_vectorizer.fit_transform(df['Title'])
lda_titles = LDA(n_components=number_topics)
lda_titles.fit(post_titles)

# fit the LDA model on all of the question bodies for the data
post_body = count_vectorizer.fit_transform(df['Title'])
lda_body = LDA(n_components=number_topics)
lda_body.fit(post_body)

LatentDirichletAllocation()

In [93]:
# print the top topics for titles
print("Post Titles: Top topics")
print()
print_topics(lda_titles, count_vectorizer, number_words)

Post Titles: Top topics


Topic 0: tag | question | programmers | stack | request | close | merge | synonym | overflow | questions
Topic 1: question | answers | questions | migrated | wrong | review | answer | closed | moderators | accepted
Topic 2: question | topic | questions | answer | lock | blacklist | hold | broad | bad | historical
Topic 3: answer | deleted | question | use | comment | blog | programmers | code | posts | post
Topic 4: reputation | question | migration | questions | place | answer | flag | voted | delete | right
Topic 5: questions | ask | software | topic | based | engineering | question | opinion | programming | asking
Topic 6: questions | site | faq | closed | tag | quality | user | programmers | discussion | bad
Topic 7: edit | community | moderator | wiki | topic | election | don | chat | questions | programmers
Topic 8: question | closed | topic | site | vote | programmers | reopen | did | software | ask
Topic 9: question | questions | career | closed | dupl

In [94]:
# print the top topics for post contents
print("Post Content: Top topics")
print_topics(lda_body, count_vectorizer, number_words)

Post Content: Top topics

Topic 0: question | closed | duplicate | vote | questions | reopen | close | deleted | reputation | user
Topic 1: question | se | programmers | topic | answers | make | improve | does | locked | tag
Topic 2: programmers | stack | questions | close | overflow | meta | new | exchange | help | users
Topic 3: site | community | question | code | blog | programmers | comments | broad | migration | election
Topic 4: based | tag | opinion | question | career | blacklist | need | cleanup | votes | ask
Topic 5: tag | question | answer | request | merge | synonym | wrong | answers | deleted | flag
Topic 6: question | questions | review | bad | best | edit | subjective | people | don | programmers
Topic 7: question | ask | programming | closed | questions | programmers | link | place | right | answer
Topic 8: questions | topic | badge | site | design | career | tags | closed | software | programmers
Topic 9: questions | software | question | engineering | topic | closing

In [95]:
# display the topic mappings with top words for 'titles'
visual_lda_titles = sklearn_lda.prepare(lda_titles,post_titles, count_vectorizer)
pyLDAvis.display(visual_lda_titles)

In [84]:
# display the topic mappings with top words for 'post contents'
visual_lda_body = sklearn_lda.prepare(lda_body,post_body, count_vectorizer)
pyLDAvis.display(visual_lda_body)